In [1]:
%pip install --upgrade keras keras_preprocessing

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import os
import re


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer



# SVM
from sklearn.svm import LinearSVC  # for text, linear SVM usually works best
from sklearn.svm import SVC

import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# LSTM (use "Base" conda env)
import tensorflow as tf
from keras.models import Sequential 
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer
# from keras._tf_keras.keras.models import Sequential
# from keras._tf_keras.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix



nltk.download('punkt_tab')
#nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/abigailcalderon/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/abigailcalderon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abigailcalderon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/abigailcalderon/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


This notebook aims to train a model on the most reliable dataset so that it can reclassify HateSpeechDatasetBalanced.csv since it's very inconsistent

In [4]:
df1 = pd.read_csv("TrainingData/labeled_data.csv")
df2 = pd.read_csv("TrainingData/HateSpeechDatasetBalanced.csv").sample(n=3000,random_state=42)


df1 = df1.dropna()
df1 = df1.drop_duplicates()
df1= df1.reset_index(drop=True)
df1

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [40]:
hate = df1[df1['class'] == 0] #1430
non_hate = df1[df1['class'] == 2].sample(n=2100, random_state=42)# 1200
offensive = df1[df1['class'] == 1].sample(n=500, random_state=42) #600

hate2 = df2[df2['Label'] == 1].sample(n=900, random_state=42) # 500

df = pd.concat([hate, non_hate, offensive], axis=0)
df = df.reset_index(drop=True)
df['class']= df['class'].replace({0: 1, 1: 0, 2: 0})
df['Content'] = df['tweet']
df['Label'] = df['class']
df = df[['Content', 'Label']]
df = pd.concat([df, hate2], axis=0)
df['Label'].value_counts()


Label
0    2600
1    2330
Name: count, dtype: int64

In [41]:
training_data_df = df.copy()
# training_data_df['Label'] = training_data_df['class']
# training_data_df['Content'] = training_data_df['tweet'].astype(str)
training_data_df = training_data_df[['Content', 'Label']]
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english')) #stopwords

def clean_tokenize(text):
    tokens = word_tokenize(text)  # splits into words and keeps contractions
    tokens = [t for t in tokens if t.isalpha()]  # keep only alphabetic tokens
    tokens = [t for t in tokens if t != "rt" and t not in stop_words]  # remove 'rt' and stopwords
    lemmatized = [lemmatizer.lemmatize(t) for t in tokens]
    return lemmatized


training_data_df['cleaned_tokens'] = training_data_df['Content'].apply(clean_tokenize)
training_data_df['tokenized_cleaned_text'] = training_data_df['cleaned_tokens'].apply(lambda x: ' '.join(x))
training_data_df['tokenized_cleaned_text'] = training_data_df['tokenized_cleaned_text'].fillna('')

# print(training_data_df.head())

tfidf = TfidfVectorizer(ngram_range=(1,2))      # playing around with ngram_range, might go up to 3

X = tfidf.fit_transform(training_data_df['tokenized_cleaned_text'])  # Features
y = training_data_df['Label']  # Target labels

print("TF-IDF shape:", X.shape)
# print(tfidf.get_feature_names_out()[:100])


# SVM training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y, shuffle=True)
model = LinearSVC(C=1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

legend = {0: "' is permitted", 1: "' has been flagged for hate speech."}


def moderate(tweet):
    tweet_clean = " ".join(clean_tokenize(tweet))
    tweet_vec = tfidf.transform([tweet_clean])
    pred = model.predict(tweet_vec)[0]
    print("The tweet: '",tweet,legend[pred],". Value: ",pred)
    
    
moderate("stfu you piece of trash")  # 1
moderate("Hope you're having a good day you cute piece of shit")  # 0
moderate("hello there")
moderate("today was fine")

training_data_df

TF-IDF shape: (4930, 51582)
[[474  46]
 [ 85 381]]
              precision    recall  f1-score   support

           0       0.85      0.91      0.88       520
           1       0.89      0.82      0.85       466

    accuracy                           0.87       986
   macro avg       0.87      0.86      0.87       986
weighted avg       0.87      0.87      0.87       986

The tweet: ' stfu you piece of trash ' is permitted . Value:  0
The tweet: ' Hope you're having a good day you cute piece of shit ' has been flagged for hate speech. . Value:  1
The tweet: ' hello there ' has been flagged for hate speech. . Value:  1
The tweet: ' today was fine ' is permitted . Value:  0


,Content,Label,cleaned_tokens,tokenized_cleaned_text
0,"""@Blackman38Tide: @WhaleLookyHere @HowdyDowdy1...",1,"[WhaleLookyHere, queer, gaywad]",WhaleLookyHere queer gaywad
1,"""@CB_Baby24: @white_thunduh alsarabsss"" hes a ...",1,"[alsarabsss, he, beaner, smh, tell, he, mexican]",alsarabsss he beaner smh tell he mexican
2,"""@DevilGrimz: @VigxRArts you're fucking gay, b...",1,"[DevilGrimz, VigxRArts, fucking, gay, blacklis...",DevilGrimz VigxRArts fucking gay blacklisted h...
3,"""@MarkRoundtreeJr: LMFAOOOO I HATE BLACK PEOPL...",1,"[MarkRoundtreeJr, LMFAOOOO, I, HATE, BLACK, PE...",MarkRoundtreeJr LMFAOOOO I HATE BLACK PEOPLE h...
4,"""@NoChillPaz: ""At least I'm not a nigger"" http...",1,"[NoChillPaz, At, least, I, nigger, http, Lmfao]",NoChillPaz At least I nigger http Lmfao
...,...,...,...,...
710738,and do non delete things from my talk asshole,1,"[non, delete, thing, talk, asshole]",non delete thing talk asshole
97844,being gay is a fate worse than death,1,"[gay, fate, worse, death]",gay fate worse death
712804,deaf to norwegians and all manner semitic acts...,1,"[deaf, norwegian, manner, semitic, act, collab...",deaf norwegian manner semitic act collaborated...
715360,when having been easy for liberals through the...,1,"[easy, liberal, obama, administration, liberal...",easy liberal obama administration liberal scre...


In [42]:
def clean_for_lstm(text):
    text = re.sub(r'(rt)?\s?@\w+:?', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^A-Za-z\'\s]', '', text)

    #remove "rt"
    text = re.sub(r'RT', '', text)
    return text.lower().strip()

# clean_tweet = tweet_column.str.replace(r'(rt)?\s?@\w+:?', ' ', regex=True).str.replace(r'http.+', ' ', regex=True).str.replace(r'\W+', ' ', regex=True)
training_data_df['cleaned_text'] = training_data_df['Content'].apply(clean_for_lstm)
training_data_df.head()




# LSTM requires sequences of integers, NOT vectors
texts = training_data_df['cleaned_text'].values
labels = training_data_df['Label'].values

# Tokenize
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)

sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, padding='post', maxlen=50)  # 50 = max length

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42, stratify=labels)
# model = Sequential([
#     Embedding(input_dim=10000, output_dim=64, input_length=50),
#     Bidirectional(LSTM(64, return_sequences=False)),
#     Dropout(0.5),
#     #Dense(16, activation='softmax'),
#     Dense(4, activation='relu'),
#     # Dense(4, activation='softmax'),
#     Dense(1, activation='sigmoid')
# ])
model = Sequential([
    Embedding(input_dim=10000, output_dim=32, input_length=50),
    Bidirectional(LSTM(32, return_sequences=False)),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

history = model.fit(X_train, y_train,
                    epochs=5, # 7 epochs: 77% , 5 epochs had 77% accuracy, 3 epochs has 79%
                    batch_size=64,
                    validation_split=0.1)

y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


legend = {0: "' is permitted", 1: "' has been flagged for hate speech."}

def moderate_lstm(tweet):
    cleaned = clean_for_lstm(tweet)
    print(cleaned)
    seq = tokenizer.texts_to_sequences([cleaned])
    padded = pad_sequences(seq, padding='post', maxlen=50)
    pred = model.predict(padded)[0][0]
    xx = lambda x : 1 if x > 0.7 else 0
    print('The tweet: "',tweet,legend[xx(pred)], ". Value: ",pred )
    return 1 if pred > 0.7 else 0



/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_10 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_10                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.5452 - loss: 0.6872 - val_accuracy: 0.6329 - val_loss: 0.6136
Epoch 2/5
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7359 - loss: 0.5636 - val_accuracy: 0.7367 - val_loss: 0.5520
Epoch 3/5
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.8015 - loss: 0.4772 - val_accuracy: 0.7873 - val_loss: 0.4638
Epoch 4/5
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.8759 - loss: 0.3182 - val_accuracy: 0.8203 - val_loss: 0.3953
Epoch 5/5
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9210 - loss: 0.2170 - val_accuracy: 0.8253 - val_loss: 0.3771
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
              precision    recall  f1-score   support

           0       0.78      0.93      0.85       520
           1       0.91      0.71      0.80       466

    accuracy                           0.83       986
   macro avg       0.84      0.82      0.82       986
weighted avg       0.84      0.83      0.83       986

[[48

In [43]:
moderate_lstm("hello there")
moderate_lstm("today was fine")
moderate_lstm("twirl")
moderate_lstm("stfu you piece of trash")  # 1
moderate_lstm("Hope you're having a good day you cute piece of shity")
moderate_lstm("I love you")
moderate_lstm("I hate you")
moderate_lstm("I hate it here")
moderate_lstm("women shouldnt vote")
moderate_lstm("yellow people are useless")

hello there
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
The tweet: " hello there ' is permitted . Value:  0.069783695
today was fine
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
The tweet: " today was fine ' is permitted . Value:  0.052633047
twirl
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
The tweet: " twirl ' has been flagged for hate speech. . Value:  0.76038206
stfu you piece of trash
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
The tweet: " stfu you piece of trash ' is permitted . Value:  0.3456063
hope you're having a good day you cute piece of shity
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
The tweet: " Hope you're having a good day you cute piece of shity ' has been flagged for hate speech. . Value:  0.99445087
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
The tweet: " I love you ' is permitted . Value:  0.08799138
i hate you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
The tweet: " I hate you ' is permitted . Value:  0.28928092
i hate it here
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
The tweet: " I hate it here '

0

In [44]:

from keras.preprocessing.sequence import pad_sequences

noisy_df = pd.read_csv("TrainingData/HateSpeechDatasetBalanced.csv")
# Clean the text
noisy_df['cleaned'] = noisy_df['Content'].apply(clean_for_lstm)

# Tokenize and pad
sequences = tokenizer.texts_to_sequences(noisy_df['cleaned'])
padded = pad_sequences(sequences, maxlen=50, padding='post')

# Predict probabilities
probs = model.predict(padded).flatten()

# Create prediction labels based on thresholds
noisy_df['pred_prob'] = probs
noisy_df['predicted_label'] = np.where(probs > 0.8, 1,
                              np.where(probs < 0.2, 0, -1))  # -1 = unsure

# Filter to high-confidence only
confident_df = noisy_df[noisy_df['predicted_label'] != -1].copy()


22692/22692 ━━━━━━━━━━━━━━━━━━━━ 78s 3ms/step


In [ ]:
confident_df

In [51]:
confident_df

#filter out df so that labeled hate speech is predicted with 98% accuracy and both label and predicted labbel are 1
hate_df = confident_df[confident_df['predicted_label'] == 1]
hate_df = hate_df[hate_df['Label'] == 1]
hate_df = hate_df[hate_df['pred_prob'] > 0.977].sample(n=16000, random_state=4)

nonhate_df = confident_df[confident_df['predicted_label'] == 0]
nonhate_df = nonhate_df[nonhate_df['Label'] == 0]
nonhate_df = nonhate_df[nonhate_df['pred_prob'] < 0.067]

combined_df = pd.concat([hate_df, nonhate_df], axis=0)
combined_df.to_csv("TrainingData/combined_df_2.csv", index=False)



In [ ]:
hate_df
nonhate_df
hate_df

,Content,Label,cleaned,pred_prob,predicted_label
529,now with over downloads,0,now with over downloads,0.053934,0
764,i had actually written the post for last summe...,0,i had actually written the post for last summe...,0.037018,0
1088,hungry horse see falls,0,hungry horse see falls,0.053767,0
1149,nothing yet eh,0,nothing yet eh,0.046162,0
1216,red hot and smoking,0,red hot and smoking,0.055901,0
...,...,...,...,...,...
440031,to look over repeatedly,0,to look over repeatedly,0.050050,0
440273,sorry i forgot to sign earlier i am new,0,sorry i forgot to sign earlier i am new,0.052000,0
440366,i have arrived just in time i have arrived jus...,0,i have arrived just in time i have arrived jus...,0.045318,0
440591,agreed i had that part wrong,0,agreed i had that part wrong,0.055324,0
